## Libraries & Daily return function

In [83]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import math

In [84]:
stocks = pd.read_csv("Portfolio.csv", index_col = "Date")

In [85]:
# Calculate daily returns
def daily_returns(adjusted_cp):
    r = (adjusted_cp / adjusted_cp.shift(1)) - 1
    return r


## Variance-Covariance Method

In [86]:
def calcuate_VaR_cov_method(portfolio, w, total_portfolio):
    daily_return_df = portfolio.apply(daily_returns).dropna()
    weights = np.array(w)
    portfolio_return = (daily_return_df * weights).sum(axis = 1)
    portfolio_mean = portfolio_return.mean()
    weights = weights.reshape(len(w),1)
    cov_matrix = daily_return_df.cov()
    portfolio_sd = np.sqrt(weights.T @ cov_matrix @ weights)[0][0]
    VaR99 = round((portfolio_mean + norm.ppf(0.01) * portfolio_sd) * total_portfolio,0)
    VaR95 = round((portfolio_mean + norm.ppf(0.05) * portfolio_sd) * total_portfolio,0)
    VaR90 = round((portfolio_mean + norm.ppf(0.10) * portfolio_sd) * total_portfolio,0)
    CVaR99 = round(portfolio_mean - (portfolio_sd / (0.01))*norm.pdf(norm.ppf(0.01)) * total_portfolio,0)
    CVaR95 = round(portfolio_mean - (portfolio_sd / (0.05))*norm.pdf(norm.ppf(0.05)) * total_portfolio,0)
    CVaR90 = round(portfolio_mean - (portfolio_sd / (0.1))*norm.pdf(norm.ppf(0.1)) * total_portfolio,0)
    print("For 1 day trading interval VaR(99%) is $", VaR99)
    print("For 1 day trading interval VaR(95%) is $", VaR95)
    print("For 1 day trading interval VaR(90%) is $", VaR90)
    print("For 1 day trading interval CVaR(99%) is $", CVaR99)
    print("For 1 day trading interval CVaR(95%) is $", CVaR95)
    print("For 1 day trading interval CVaR(90%) is $", CVaR90)
calcuate_VaR_cov_method(stocks, w = [0.3,0.4,0.3], total_portfolio = 1000000)    
    

For 1 day trading interval VaR(99%) is $ -35767.0
For 1 day trading interval VaR(95%) is $ -25073.0
For 1 day trading interval VaR(90%) is $ -19372.0
For 1 day trading interval CVaR(99%) is $ -41821.0
For 1 day trading interval CVaR(95%) is $ -32367.0
For 1 day trading interval CVaR(90%) is $ -27538.0


## Historical Method

In [87]:
def calcuate_VaR_hist_method(portfolio, w, total_portfolio):
    daily_return_df = portfolio.apply(daily_returns).dropna()
    weights = np.array(w)
    portfolio_return = (daily_return_df * weights).sum(axis = 1)
    VaR99 = round(np.percentile(portfolio_return,1) * total_portfolio,0)
    VaR95 = round(np.percentile(portfolio_return,5) * total_portfolio,0)
    VaR90 = round(np.percentile(portfolio_return,10) * total_portfolio,0)
    CVaR99 = round(np.mean(portfolio_return[portfolio_return < np.percentile(portfolio_return,1)]) * total_portfolio,0)
    CVaR95 = round(np.mean(portfolio_return[portfolio_return < np.percentile(portfolio_return,5)]) * total_portfolio,0)
    CVaR90 = round(np.mean(portfolio_return[portfolio_return < np.percentile(portfolio_return,10)]) * total_portfolio,0)
    print("For 1 day trading interval VaR(99%) is $", VaR99)
    print("For 1 day trading interval VaR(95%) is $", VaR95)
    print("For 1 day trading interval VaR(90%) is $", VaR90)
    print("For 1 day trading interval CVaR(99%) is $", CVaR99)
    print("For 1 day trading interval CVaR(95%) is $", CVaR95)
    print("For 1 day trading interval CVaR(90%) is $", CVaR90)
    
calcuate_VaR_hist_method(stocks, w = [0.3,0.4,0.3], total_portfolio = 1000000)

For 1 day trading interval VaR(99%) is $ -35630.0
For 1 day trading interval VaR(95%) is $ -23896.0
For 1 day trading interval VaR(90%) is $ -19206.0
For 1 day trading interval CVaR(99%) is $ -41297.0
For 1 day trading interval CVaR(95%) is $ -31545.0
For 1 day trading interval CVaR(90%) is $ -26954.0


## Monte Carlo Simulation method 

$ R_t \sim MVN(\mu, \Sigma)$  
Cholesky decomposition  
$LL' = \Sigma$  
$R_t = \mu + LZ^{T}$  
$Z \sim N(0,I)$ (I is Indentity Matrix)


In [120]:
meanReturns = returns.mean()
num_stocks = stocks.shape[1]
covMatrix = returns.cov()
weights = np.array([0.40, 0.30, 0.30])
mc_sims = 100
T = 252 # time frame in days
portfolio_value = 1000000
meanM = np.full(shape=(T, num_stocks), fill_value=meanReturns)
meanM = meanM.T
simulated_portfolio_returns = np.full(shape = (T, mc_sims), fill_value=0.0)

for m in range(0, mc_sims):
    Z = np.random.normal(size = (T, num_stocks))
    L = np.linalg.cholesky(covMatrix)
    dailyReturns = meanM + L @ Z.T # l - 3*3 , z = Time*3
    simulated_portfolio_returns[:,m] = np.inner(weights, dailyReturns.T)
    
simulated_portfolio_returns = pd.DataFrame(simulated_portfolio_returns)


In [122]:
total_portfolio = 1000000
var99 = [round(np.percentile(simulated_portfolio_returns.iloc[:,i],1) * total_portfolio,0) for i in range(100)]
var95 = [round(np.percentile(simulated_portfolio_returns.iloc[:,i],5) * total_portfolio,0) for i in range(100)]
var90 = [round(np.percentile(simulated_portfolio_returns.iloc[:,i],10) * total_portfolio,0) for i in range(100)]

Cvar99 = [round(np.mean(simulated_portfolio_returns.iloc[:,i][simulated_portfolio_returns.iloc[:,i] < np.percentile(simulated_portfolio_returns.iloc[:,i],1)]) * total_portfolio,0) for i in range(100)]

Cvar95 = [round(np.mean(simulated_portfolio_returns.iloc[:,i][simulated_portfolio_returns.iloc[:,i] < np.percentile(simulated_portfolio_returns.iloc[:,i],5)]) * total_portfolio,0) for i in range(100)]

Cvar90 = [round(np.mean(simulated_portfolio_returns.iloc[:,i][simulated_portfolio_returns.iloc[:,i] < np.percentile(simulated_portfolio_returns.iloc[:,i],10)]) * total_portfolio,0) for i in range(100)]

In [123]:
def summary_statistics(var):
    return {
        "Mean" : np.mean(var),
        "Max" : max(var),
        "Min" : min(var)
    }

In [124]:
summary_statistics(var99)

{'Max': -25722.0, 'Mean': -34085.29, 'Min': -44658.0}

In [125]:
summary_statistics(var95)

{'Max': -19281.0, 'Mean': -24435.38, 'Min': -29964.0}

In [126]:
summary_statistics(var90)

{'Max': -14762.0, 'Mean': -18840.28, 'Min': -24289.0}

In [127]:
summary_statistics(Cvar90)

{'Max': -21189.0, 'Mean': -26011.59, 'Min': -32224.0}

In [128]:
summary_statistics(Cvar95)

{'Max': -25392.0, 'Mean': -30709.27, 'Min': -39025.0}

In [129]:
summary_statistics(Cvar99)

{'Max': -29077.0, 'Mean': -38694.63, 'Min': -54694.0}